### 1. Create SCENIC+ object

In [1]:
# Load functions
from scenicplus.scenicplus_class import SCENICPLUS, create_SCENICPLUS_object
from scenicplus.preprocessing.filtering import *

First we will load the scRNA-seq and the scATAC-seq data. We make sure that names match between them.

In [2]:
# Load data
## ATAC - cisTopic object
outDir = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/pycisTopic/'
import pickle
infile = open(outDir + 'DPCL_cisTopicObject.pkl', 'rb')
cistopic_obj = pickle.load(infile)
infile.close()
## Precomputed imputed data
import pickle
infile = open(outDir + 'DARs/Imputed_accessibility.pkl', 'rb')
imputed_acc_obj = pickle.load(infile)
infile.close()
## RNA - Create Anndata
from loomxpy.loomxpy import SCopeLoom
from pycisTopic.loom import *
import itertools
import anndata
path_to_loom = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/vsn/grnboost/out/data/scRNA_count_matrix.SINGLE_SAMPLE_SCENIC.loom'
loom = SCopeLoom.read_loom(path_to_loom)
metadata = get_metadata(loom)
# Fix names
metadata.index =  metadata.index + '___DPLC'
expr_mat = loom.ex_mtx
expr_mat.index =  expr_mat.index + '___DPLC'
rna_anndata = anndata.AnnData(X=expr_mat)
rna_anndata.obs = metadata

/local_scratch/tmp-vsc31305/ipykernel_10949/1356271359.py:6: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  cistopic_obj = pickle.load(infile)
/local_scratch/tmp-vsc31305/ipykernel_10949/1356271359.py:11: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  imputed_acc_obj = pickle.load(infile)


Next we load the motif enrichment results into a dictionary. We can load motif results from the different methods in pycistarget (e.g. cisTarget, DEM) and different region sets (e.g. topics, DARs, MACS bdgdiff peaks). In this tutorial we will use both cisTarget and DEM peaks from topics and DARs.

In [3]:
## Precomputed imputed data
import pickle
infile = open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/pycistarget/cluster_V10_V2/menr.pkl', 'rb') 
menr = pickle.load(infile)
infile.close()

Now we can create the SCENIC+ object:

In [4]:
scplus_obj = create_SCENICPLUS_object(
        GEX_anndata = rna_anndata,
        cisTopic_obj = cistopic_obj,
        imputed_acc_obj = imputed_acc_obj,
        menr = menr,
        ACC_prefix = 'ACC_',
        GEX_prefix = 'GEX_',
        bc_transform_func = lambda x: x,
        normalize_imputed_acc = False)

In [ ]:
type(scplus_obj.X_EXP)

In [6]:
print(scplus_obj)

SCENIC+ object with n_cells x n_genes = 4000 x 33415 and n_cells x n_regions = 4000 x 631497
	metadata_regions:'Chromosome', 'Start', 'End', 'Width', 'cisTopic_nr_frag', 'cisTopic_log_nr_frag', 'cisTopic_nr_acc', 'cisTopic_log_nr_acc'
	metadata_cell:'GEX_sample_id', 'GEX_leiden', 'ACC_cisTopic_nr_frag', 'ACC_cisTopic_log_nr_frag', 'ACC_cisTopic_nr_acc', 'ACC_cisTopic_log_nr_acc', 'ACC_sample_id', 'ACC_Log_total_nr_frag', 'ACC_Log_unique_nr_frag', 'ACC_Total_nr_frag', 'ACC_Unique_nr_frag', 'ACC_Dupl_nr_frag', 'ACC_Dupl_rate', 'ACC_Total_nr_frag_in_regions', 'ACC_Unique_nr_frag_in_regions', 'ACC_FRIP', 'ACC_TSS_enrichment', 'ACC_barcode', 'ACC_Cell_type', 'ACC_pycisTopic_leiden_10_0.3', 'ACC_Line_type'
	menr:'CTX_Topics_All', 'CTX_Topics_NP', 'CTX_DARs_All', 'CTX_DARs_NP', 'DEM_Topics_All', 'DEM_Topics_NP', 'DEM_DARs_All', 'DEM_DARs_NP'
	dr_cell:'ACC_UMAP', 'ACC_tSNE'


You can also filter low accessible regions and low expressed genes. This recommended to avoid getting false relationships with these regions and genes.

In [7]:
filter_genes(scplus_obj, min_pct = 0.5)
filter_regions(scplus_obj, min_pct = 0.5)

2022-01-10 18:27:12,010 Preprocessing INFO     Going from 33415 genes to 25651 genes.
2022-01-10 18:28:17,478 Preprocessing INFO     Going from 631497 regions to 589734 regions.


In [9]:
# Save
outDir = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/scenicplus_final/'
import pickle
with open(outDir+'scplus_obj.pkl', 'wb') as f:
  pickle.dump(scplus_obj, f)

### GRNBoost

In [ ]:
singularity exec -B /lustre1,/staging,/data,/vsc-hard-mounts,/scratch scenicplus.sif ipython3

In [ ]:
# For the downstream analyses
outDir = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/scenicplus_final/'
import pickle
infile = open(outDir+'scplus_obj.pkl', 'rb')
scplus_obj = pickle.load(infile)
infile.close()

import pickle
infile = open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/scenicplus_V2_grnboost/region_ranking.pkl', 'rb')
region_ranking = pickle.load(infile)
infile.close()

import pickle
infile = open('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/scenicplus_V2_grnboost/gene_ranking.pkl', 'rb')
gene_ranking = pickle.load(infile)
infile.close()

from scenicplus.wrappers.run_scenicplus import *
run_scenicplus(scplus_obj,
    variable = ['ACC_Cell_type'],
    species = 'hsapiens',
    assembly = 'hg38',
    tf_file = '/staging/leuven/stg_00002/lcb/cflerin/resources/allTFs_hg38.txt',
    save_path = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/scenicplus_final_autoreg/grnboost/',
    biomart_host = 'http://oct2016.archive.ensembl.org/',
    upstream = [1000, 150000],
    downstream = [1000, 150000],   
    calculate_TF_eGRN_correlation = False,
    calculate_DEGs_DARs = True,
    export_to_loom_file = True,
    export_to_UCSC_file = True,
    region_ranking=region_ranking,
    gene_ranking=gene_ranking,
    tree_structure = ('DPCL', 'SCENIC+', 'grnboost'),
    path_bedToBigBed = '/data/leuven/software/biomed/haswell_centos7/2018a/software/Kent_tools/20190730-linux.x86_64/bin/',
    n_cpu = 20,
    _temp_dir = '/scratch/leuven/313/vsc31305/ray_spill'
    )

In [1]:
# For the downstream analyses
outDir = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/scenicplus_final_autoreg/grnboost/'
import dill
infile = open(outDir+'scplus_obj.pkl', 'rb')
scplus_obj = dill.load(infile)
infile.close()

In [11]:
import pandas as pd
def format_egrns_time(scplus_obj,
                 eregulons_key: str = 'eRegulons',
                 TF2G_key: str = 'TF2G_adj',
                 key_added: str = 'eRegulon_metadata'):
    """
    A function to format eRegulons to a pandas dataframe
    """

    egrn_list = scplus_obj.uns[eregulons_key]
    TF = [egrn_list[x].transcription_factor for x in range(len(egrn_list))]
    is_extended = [str(egrn_list[x].is_extended)
                   for x in range(len(egrn_list))]
    r2g_data = [pd.DataFrame.from_records(egrn_list[x].regions2genes, columns=[
                                          'Region', 'Gene', 'R2G_importance', 'R2G_rho', 'R2G_importance_x_rho', 'R2G_importance_x_abs_rho']) for x in range(len(egrn_list))]
    egrn_name = [TF[x] + '_extended' if is_extended[x] ==
                 'True' else TF[x] for x in range(len(egrn_list))]
    egrn_name = [egrn_name[x] + '_+' if 'positive tf2g' in egrn_list[x]
                 .context else egrn_name[x] + '_-' for x in range(len(egrn_list))]
    egrn_name = [egrn_name[x] + '_+' if 'positive r2g' in egrn_list[x]
                 .context else egrn_name[x] + '_-' for x in range(len(egrn_list))]
    region_signature_name = [
        egrn_name[x] + '_(' + str(len(set(r2g_data[x].Region))) + 'r)' for x in range(len(egrn_list))]
    gene_signature_name = [
        egrn_name[x] + '_(' + str(len(set(r2g_data[x].Gene))) + 'g)' for x in range(len(egrn_list))]

    for x in range(len(egrn_list)):
        r2g_data[x].insert(0, "TF", TF[x])
        r2g_data[x].insert(1, "is_extended", is_extended[x])
        r2g_data[x].insert(0, "Gene_signature_name", gene_signature_name[x])
        r2g_data[x].insert(0, "Region_signature_name",
                           region_signature_name[x])

    tf2g_data = scplus_obj.uns[TF2G_key].copy()
    tf2g_data.columns = ['TF', 'Gene', 'TF2G_importance', 'TF2G_regulation',
                         'TF2G_rho', 'TF2G_importance_x_abs_rho', 'TF2G_importance_x_rho']
    egrn_metadata = pd.concat([pd.merge(r2g_data[x], tf2g_data[tf2g_data.TF == r2g_data[x].TF[0]], on=[
                              'TF', 'Gene']) for x in range(len(egrn_list)) if tf2g_data[tf2g_data.TF == r2g_data[x].TF[0]].shape[0] != 0 and r2g_data[x].shape[0] != 0])
    scplus_obj.uns[key_added] = egrn_metadata

In [12]:
format_egrns_time(scplus_obj,
              eregulons_key = 'eRegulons',
              TF2G_key = 'TF2G_adj',
              key_added = 'eRegulon_metadata')

In [4]:
scplus_obj.uns['region_to_gene'].to_csv('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/scenicplus_V2_grnboost_2/region_to_gene.tsv', sep='\t')



In [ ]:
scplus_obj.uns['region_to_gene'][scplus_obj.uns['region_to_gene']['rho'] >0].to_csv('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/scenicplus_V2_grnboost_2/region_to_gene_pos.tsv', sep='\t')



In [7]:
select = list(set(scplus_obj.uns['eRegulon_metadata']['Region']))

In [8]:
scplus_obj.uns['region_to_gene']

,target,region,importance,rho,importance_x_rho,importance_x_abs_rho,Distance
0,PALM3,chr19:13903093-13903594,0.007320,0.039632,0.000290,0.000290,[-150021]
1,PALM3,chr19:13916776-13917277,0.001618,-0.008657,-0.000014,0.000014,[-136339]
2,PALM3,chr19:13918892-13919393,0.007048,0.097071,0.000684,0.000684,[-134223]
3,PALM3,chr19:13924873-13925374,0.000756,0.073550,0.000056,0.000056,[-128241]
4,PALM3,chr19:13929165-13929666,0.002574,-0.079550,-0.000205,0.000205,[-123949]
...,...,...,...,...,...,...,...
2083810,STMN4,chr8:27400009-27400510,0.029715,-0.025321,-0.000752,0.000752,[141840]
2083811,STMN4,chr8:27404043-27404544,0.004020,-0.004116,-0.000017,0.000017,[145874]
2083812,STMN4,chr8:27404765-27405266,0.002344,0.177206,0.000415,0.000415,[146596]
2083813,STMN4,chr8:27405285-27405786,0.019289,0.195575,0.003772,0.003772,[147116]


In [9]:
scplus_obj.uns['region_to_gene'][scplus_obj.uns['region_to_gene']['region'].isin(select)]

,target,region,importance,rho,importance_x_rho,importance_x_abs_rho,Distance
12,PALM3,chr19:13937615-13938116,0.005005,-0.070210,-0.000351,0.000351,[-115499]
13,PALM3,chr19:13938255-13938756,0.005120,-0.055077,-0.000282,0.000282,[-114859]
19,PALM3,chr19:13958022-13958523,0.008706,0.088218,0.000768,0.000768,[-95093]
24,PALM3,chr19:13962328-13962829,0.019312,-0.119996,-0.002317,0.002317,[-90787]
26,PALM3,chr19:13964715-13965216,0.005130,-0.121418,-0.000623,0.000623,[-88399]
...,...,...,...,...,...,...,...
2083786,STMN4,chr8:27363649-27364150,0.028793,0.137378,0.003956,0.003956,[105480]
2083787,STMN4,chr8:27365988-27366489,0.033363,0.189441,0.006320,0.006320,[107818]
2083788,STMN4,chr8:27366506-27367007,0.053182,0.124266,0.006609,0.006609,[108336]
2083796,STMN4,chr8:27378607-27379108,0.044126,0.199834,0.008818,0.008818,[120438]


In [1]:
scplus_obj.uns['region_to_gene'][scplus_obj.uns['region_to_gene']['region'].isin(select)].to_csv('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/scenicplus_V2_grnboost_2/region_to_gene_in_eGRN.tsv', sep='\t')

NameError: name 'scplus_obj' is not defined

In [63]:
import pandas as pd
hic_data = pd.read_csv('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/formatted/HepG2_ENCFF020DPP_5Kb_SCALE.txt', sep='\t')
hic_data.columns = ['Chromosome', 'Start', 'End', 'target', 'importance', 'TSS', 'Strand', 'Distance', 'region']
hic_data['rho'] = hic_data['importance']
hic_data = hic_data[['target', 'region', 'importance', 'rho', 'Distance']]
hic_data = hic_data[hic_data['region'].isin(scplus_obj.region_names)]
hic_data 

,target,region,importance,rho,Distance
0,ZNF493,chr19:21587750-21588251,10.000017,10.000017,190631
1,ZNF493,chr19:21585171-21585672,10.000017,10.000017,188052
2,ZNF493,chr19:21586234-21586735,10.000017,10.000017,189115
3,ZNF493,chr19:21282787-21283288,10.034795,10.034795,-114332
4,ZNF493,chr19:21221605-21222106,10.049212,10.049212,-175514
...,...,...,...,...,...
2576842,NPIPA2,chr16:14680470-14680971,3856.234900,3856.234900,-67596
2576843,SLX1A,chr16:22174595-22175096,3885.789000,3885.789000,-8019280
2576844,SLX1A,chr16:22170680-22171181,3885.789000,3885.789000,-8023195
2576845,BOLA2B,chr16:22174595-22175096,3885.789000,3885.789000,8019711


In [13]:
import pandas as pd
import numpy as np
import ray
import logging
import time
import sys
import os
import subprocess
import pyranges as pr
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from scipy.stats import pearsonr, spearmanr
from tqdm import tqdm
from matplotlib import cm
from matplotlib.colors import Normalize
from typing import List

from scenicplus.utils import extend_pyranges, extend_pyranges_with_limits, reduce_pyranges_with_limits_b
from scenicplus.utils import calculate_distance_with_limits_join, reduce_pyranges_b, calculate_distance_join
from scenicplus.utils import coord_to_region_names, region_names_to_coordinates, ASM_SYNONYMS, Groupby, flatten_list
from scenicplus.scenicplus_class import SCENICPLUS
from scenicplus.enhancer_to_gene import INTERACT_AS



def export_to_UCSC_interact_hic(SCENICPLUS_obj: SCENICPLUS,
                            species: str,
                            outfile: str,
                            region_to_gene_key: str =' region_to_gene',
                            pbm_host:str = 'http://www.ensembl.org',
                            bigbed_outfile:str = None,
                            path_bedToBigBed: str= None,
                            assembly: str = None,
                            ucsc_track_name: str = 'region_to_gene',
                            ucsc_description: str = 'interaction file for region to gene',
                            cmap_neg: str = 'Reds',
                            cmap_pos: str = 'Greens',
                            key_for_color: str = 'importance',
                            vmin: int = 0,
                            vmax: int = 1,
                            scale_by_gene: bool = True,
                            subset_for_eRegulons_regions: bool = True,
                            eRegulons_key: str = 'eRegulons') -> pd.DataFrame:
    # Create logger
    level = logging.INFO
    format = '%(asctime)s %(name)-12s %(levelname)-8s %(message)s'
    handlers = [logging.StreamHandler(stream=sys.stdout)]
    logging.basicConfig(level=level, format=format, handlers=handlers)
    log = logging.getLogger('R2G')

    if region_to_gene_key not in SCENICPLUS_obj.uns.keys():
        raise Exception(
            f'key {region_to_gene_key} not found in SCENICPLUS_obj.uns, first calculate region to gene relationships using function: "calculate_regions_to_genes_relationships"')

    region_to_gene_df = SCENICPLUS_obj.uns[region_to_gene_key].copy()

    if subset_for_eRegulons_regions:
        if eRegulons_key not in SCENICPLUS_obj.uns.keys():
            raise ValueError(
                f'key {eRegulons_key} not found in SCENICPLUS_obj.uns.keys()')
        eRegulon_regions = list(set(flatten_list(
            [ereg.target_regions for ereg in SCENICPLUS_obj.uns[eRegulons_key]])))
        region_to_gene_df.index = region_to_gene_df['region']
        region_to_gene_df = region_to_gene_df.loc[eRegulon_regions].reset_index(
            drop=True)

    # Rename columns to be in line with biomart annotation
    region_to_gene_df.rename(columns={'target': 'Gene'}, inplace=True)

    # Get TSS annotation (end-point for links)
    log.info('Downloading gene annotation from biomart, using dataset: {}'.format(
        species+'_gene_ensembl'))
    import pybiomart as pbm
    dataset = pbm.Dataset(name=species+'_gene_ensembl',  host=pbm_host)
    annot = dataset.query(attributes=['chromosome_name', 'start_position', 'end_position',
                          'strand', 'external_gene_name', 'transcription_start_site', 'transcript_biotype'])
    annot.columns = ['Chromosome', 'Start', 'End', 'Strand',
                     'Gene', 'Transcription_Start_Site', 'Transcript_type']
    annot['Chromosome'] = 'chr' + \
        annot['Chromosome'].astype(str)
    annot = annot[annot.Transcript_type == 'protein_coding']
    annot.Strand[annot.Strand == 1] = '+'
    annot.Strand[annot.Strand == -1] = '-'

    log.info('Formatting data ...')
    # get gene to tss mapping, take the one equal to the gene start/end location if possible otherwise take the first one
    annot['TSSeqStartEnd'] = np.logical_or(
        annot['Transcription_Start_Site'] == annot['Start'], annot['Transcription_Start_Site'] == annot['End'])
    gene_to_tss = annot[['Gene', 'Transcription_Start_Site']].groupby(
        'Gene').agg(lambda x: list(map(str, x)))
    startEndEq = annot[['Gene', 'TSSeqStartEnd']
                       ].groupby('Gene').agg(lambda x: list(x))
    gene_to_tss['Transcription_Start_Site'] = [np.array(tss[0])[eq[0]][0] if sum(
        eq[0]) >= 1 else tss[0][0] for eq, tss in zip(startEndEq.values, gene_to_tss.values)]
    gene_to_tss.columns = ['TSS_Gene']

    # get gene to strand mapping
    gene_to_strand = annot[['Gene', 'Strand']].groupby(
        'Gene').agg(lambda x: list(map(str, x))[0])

    # get gene to chromosome mapping (should be the same as the regions mapped to the gene)
    gene_to_chrom = annot[['Gene', 'Chromosome']].groupby(
        'Gene').agg(lambda x: list(map(str, x))[0])

    # add TSS for each gene to region_to_gene_df
    region_to_gene_df = region_to_gene_df.join(gene_to_tss, on='Gene')

    # add strand for each gene to region_to_gene_df
    region_to_gene_df = region_to_gene_df.join(gene_to_strand, on='Gene')

    # add chromosome for each gene to region_to_gene_df
    region_to_gene_df = region_to_gene_df.join(gene_to_chrom, on='Gene')

    # get chrom, chromStart, chromEnd
    region_to_gene_df.dropna(axis=0, how='any', inplace=True)
    arr = region_names_to_coordinates(region_to_gene_df['region']).to_numpy()
    chrom, chromStart, chromEnd = np.split(arr, 3, 1)
    chrom = chrom[:, 0]
    chromStart = chromStart[:, 0]
    chromEnd = chromEnd[:, 0]

    # get source chrom, chromStart, chromEnd (i.e. middle of regions)
    sourceChrom = chrom
    sourceStart = np.array(
        list(map(int, chromStart + (chromEnd - chromStart)/2 - 1)))
    sourceEnd = np.array(
        list(map(int, chromStart + (chromEnd - chromStart)/2)))

    # get target chrom, chromStart, chromEnd (i.e. TSS)
    targetChrom = region_to_gene_df['Chromosome']
    targetStart = region_to_gene_df['TSS_Gene'].values
    targetEnd = list(map(str, np.array(list(map(int, targetStart))) + np.array(
        [1 if strand == '+' else -1 for strand in region_to_gene_df['Strand'].values])))

    # get color
    norm = Normalize(vmin=vmin, vmax=vmax)
    if scale_by_gene:
        grouper = Groupby(
            region_to_gene_df.loc[:, 'Gene'].to_numpy())
        scores = region_to_gene_df.loc[:, key_for_color].to_numpy()
        mapper = cm.ScalarMappable(norm=norm, cmap=cmap_pos)

        def _value_to_color(scores):
            S = (scores - scores.min()) / (scores.max() - scores.min())
            return [','.join([str(x) for x in mapper.to_rgba(s, bytes=True)][0:3]) for s in S]

        colors_pos = np.zeros(len(scores), dtype='object')
        for idx in grouper.indices:
            colors_pos[idx] = _value_to_color(scores[idx])

        def _value_to_color(scores):
            S = (scores - scores.min()) / (scores.max() - scores.min())
            return [','.join([str(x) for x in mapper.to_rgba(s, bytes=True)][0:3]) for s in S]

        colors_neg = np.zeros(len(scores), dtype='object')
        for idx in grouper.indices:
            colors_neg[idx] = _value_to_color(scores[idx])

    else:
        scores = region_to_gene_df.loc[:, key_for_color].to_numpy()
        mapper = cm.ScalarMappable(norm=norm, cmap=cmap_pos)
        colors_pos = [
            ','.join([str(x) for x in mapper.to_rgba(s, bytes=True)][0:3]) for s in scores]


    region_to_gene_df.loc[:, 'color'] = colors_pos
    region_to_gene_df['color'] = region_to_gene_df['color'].fillna('55,55,55')

    # get name for regions (add incremental number to gene in range of regions linked to gene)
    counter = 1
    previous_gene = region_to_gene_df['Gene'].values[0]
    names = []
    for gene in region_to_gene_df['Gene'].values:
        if gene != previous_gene:
            counter = 1
        else:
            counter += 1
        names.append(gene + '_' + str(counter))
        previous_gene = gene

    # format final interact dataframe
    df_interact = pd.DataFrame(
        data={
            'chrom':        chrom,
            'chromStart':   chromStart,
            'chromEnd':     chromEnd,
            'name':         names,
            'score':        (1000*(region_to_gene_df['importance'].values - np.min(region_to_gene_df['importance'].values))/np.ptp(region_to_gene_df['importance'].values)).astype(int) ,
            'value':        region_to_gene_df['importance'].values,
            'exp':          np.repeat('.', len(region_to_gene_df)),
            'color':        region_to_gene_df['color'].values,
            'sourceChrom':  sourceChrom,
            'sourceStart':  sourceStart,
            'sourceEnd':    sourceEnd,
            'sourceName':   names,
            'sourceStrand': np.repeat('.', len(region_to_gene_df)),
            'targetChrom':  targetChrom,
            'targetStart':  targetStart,
            'targetEnd':    targetEnd,
            'targetName':   region_to_gene_df['Gene'].values,
            'targetStrand': region_to_gene_df['Strand'].values
        }
    )
    # sort dataframe
    df_interact = df_interact.sort_values(by=['chrom', 'chromStart'])
    # Write interact file
    log.info('Writing data to: {}'.format(outfile))
    with open(outfile, 'w') as f:
        f.write('track type=interact name="{}" description="{}" useScore=0 maxHeightPixels=200:100:50 visibility=full\n'.format(
            ucsc_track_name, ucsc_description))
        df_interact.to_csv(f, header=False, index=False, sep='\t')

    # write bigInteract file
    if bigbed_outfile != None:
        log.info('Writing data to: {}'.format(bigbed_outfile))
        outfolder = bigbed_outfile.rsplit('/', 1)[0]
        # write bed file without header to tmp file
        df_interact.to_csv(os.path.join(
            outfolder, 'interact.bed.tmp'), header=False, index=False, sep='\t')

        # check if auto sql definition for interaction file exists in outfolder, otherwise create it
        if not os.path.exists(os.path.join(outfolder, 'interact.as')):
            with open(os.path.join(outfolder, 'interact.as'), 'w') as f:
                f.write(INTERACT_AS)
        # convert interact.bed.tmp to bigBed format
        # bedToBigBed -as=interact.as -type=bed5+13 region_to_gene_no_head.interact https://genome.ucsc.edu/goldenPath/help/hg38.chrom.sizes region_to_gene.inter.bb
        cmds = [
            os.path.join(path_bedToBigBed, 'bedToBigBed'),
            '-as={}'.format(os.path.join(os.path.join(outfolder, 'interact.as'))),
            '-type=bed5+13',
            os.path.join(outfolder, 'interact.bed.tmp'),
            'https://hgdownload.cse.ucsc.edu/goldenpath/' + assembly + '/bigZips/' + assembly + '.chrom.sizes',
            bigbed_outfile
        ]
        p = subprocess.Popen(cmds, stdout=subprocess.PIPE,
                             stderr=subprocess.PIPE)
        stdout, stderr = p.communicate()
        if p.returncode:
            raise ValueError(
                "cmds: %s\nstderr:%s\nstdout:%s" % (
                    " ".join(cmds), stderr, stdout)
            )
    return df_interact

# Generate Hi-C files

In [18]:
# For the downstream analyses
outDir = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/scenicplus_final_autoreg/grnboost/'
import dill
infile = open(outDir+'scplus_obj.pkl', 'rb')
scplus_obj = dill.load(infile)
infile.close()

In [21]:
r2g = scplus_obj.uns['region_to_gene'].copy()
r2g = r2g[r2g.rho > 0.03]
r2g

,target,region,importance,rho,importance_x_rho,importance_x_abs_rho,Distance
0,PALM3,chr19:13903093-13903594,0.007320,0.039632,0.000290,0.000290,[-150021]
2,PALM3,chr19:13918892-13919393,0.007048,0.097071,0.000684,0.000684,[-134223]
3,PALM3,chr19:13924873-13925374,0.000756,0.073550,0.000056,0.000056,[-128241]
9,PALM3,chr19:13934093-13934594,0.003016,0.046918,0.000142,0.000142,[-119021]
14,PALM3,chr19:13940538-13941039,0.010659,0.113388,0.001209,0.001209,[-112577]
...,...,...,...,...,...,...,...
2083802,STMN4,chr8:27388148-27388649,0.009306,0.204597,0.001904,0.001904,[129978]
2083806,STMN4,chr8:27393594-27394095,0.028735,0.188804,0.005425,0.005425,[135424]
2083807,STMN4,chr8:27394954-27395455,0.016020,0.032904,0.000527,0.000527,[136784]
2083812,STMN4,chr8:27404765-27405266,0.002344,0.177206,0.000415,0.000415,[146596]


In [22]:
scplus_obj.uns['region_to_gene'].to_csv('/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/scenicplus_final_autoreg/grnboost/region_to_gene_pos.tsv', sep='\t')

In [ ]:
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
cell_lines=['GM12878', 'HepG2', 'IMR90', 'HCT116', 'K562']
save_path = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/bb_files_final/'
species = 'hsapiens'
assembly = 'hg38'
path_bedToBigBed = '/data/leuven/software/biomed/haswell_centos7/2018a/software/Kent_tools/20190730-linux.x86_64/bin/'
biomart_host = 'http://oct2016.archive.ensembl.org/'
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/formatted/'
for cell_line in cell_lines:
    files = [f for f in listdir(path) if isfile(join(path, f))]
    file = [f for f in files if cell_line in f]
    hic_data = pd.read_csv(path+file[0], sep='\t')
    hic_data.columns = ['Chromosome', 'Start', 'End', 'target', 'importance', 'TSS', 'Strand', 'Distance', 'region']
    hic_data['rho'] = hic_data['importance']
    hic_data = hic_data[['target', 'region', 'importance', 'rho', 'Distance']]
    hic_data = hic_data[hic_data['region'].isin(scplus_obj.region_names)]
    scplus_obj.uns['region_to_gene'] = hic_data
    r2g_data = export_to_UCSC_interact_hic(scplus_obj,
                    species,
                    os.path.join(save_path,cell_line+'.hic.all.bed'),
                    path_bedToBigBed=path_bedToBigBed,
                    bigbed_outfile=os.path.join(save_path,cell_line+'.hic.all.bb'),
                    region_to_gene_key='region_to_gene',
                    pbm_host=biomart_host,
                    assembly=assembly,
                    ucsc_track_name='R2G',
                    ucsc_description=cell_line+' HiC links',
                    cmap_neg='Reds',
                    cmap_pos='Greys',
                    key_for_color='importance',
                    scale_by_gene=True,
                    subset_for_eRegulons_regions=False,
                    eRegulons_key='eRegulons')

In [17]:
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
cell_lines=['CellOracle', 'FigR', 'GRaNIE', 'Scenicplus-importance', 'Scenicplus-rho']
color_dict={'CellOracle':'Greens', 'FigR':'Purples', 'GRaNIE':'Oranges', 'Scenicplus-importance':'Blues', 'Scenicplus-rho':'Reds'}
save_path = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/bb_files_final/'
species = 'hsapiens'
assembly = 'hg38'
path_bedToBigBed = '/data/leuven/software/biomed/haswell_centos7/2018a/software/Kent_tools/20190730-linux.x86_64/bin/'
biomart_host = 'http://oct2016.archive.ensembl.org/'
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/formatted/'
for cell_line in cell_lines:
    files = [f for f in listdir(path) if isfile(join(path, f))]
    file = [f for f in files if cell_line in f]
    hic_data = pd.read_csv(path+file[0], sep='\t')
    hic_data.columns = ['Chromosome', 'Start', 'End', 'target', 'importance', 'TSS', 'Strand', 'Distance', 'region']
    hic_data['rho'] = hic_data['importance']
    hic_data = hic_data[['target', 'region', 'importance', 'rho', 'Distance']]
    hic_data = hic_data[hic_data['region'].isin(scplus_obj.region_names)]
    scplus_obj.uns['region_to_gene'] = hic_data
    r2g_data = export_to_UCSC_interact_hic(scplus_obj,
                    species,
                    os.path.join(save_path,cell_line+'.links.all.bed'),
                    path_bedToBigBed=path_bedToBigBed,
                    bigbed_outfile=os.path.join(save_path,cell_line+'.links.all.bb'),
                    region_to_gene_key='region_to_gene',
                    pbm_host=biomart_host,
                    assembly=assembly,
                    ucsc_track_name='R2G',
                    ucsc_description='SCENIC+ region to gene links',
                    cmap_neg='Reds',
                    cmap_pos=color_dict[cell_line],
                    key_for_color='importance',
                    scale_by_gene=True,
                    subset_for_eRegulons_regions=False,
                    eRegulons_key='eRegulons')


2022-05-23 13:32:34,106 R2G          INFO     Downloading gene annotation from biomart, using dataset: hsapiens_gene_ensembl
2022-05-23 13:32:34,679 R2G          INFO     Formatting data ...


/tmp/ipykernel_29552/570429411.py:143: RuntimeWarning: invalid value encountered in true_divide
  S = (scores - scores.min()) / (scores.max() - scores.min())
/tmp/ipykernel_29552/570429411.py:151: RuntimeWarning: invalid value encountered in true_divide
  S = (scores - scores.min()) / (scores.max() - scores.min())


2022-05-23 13:34:40,211 R2G          INFO     Writing data to: /staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/bb_files_final/CellOracle.links.all.bed
2022-05-23 13:34:42,981 R2G          INFO     Writing data to: /staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/bb_files_final/CellOracle.links.all.bb
2022-05-23 13:34:51,182 R2G          INFO     Downloading gene annotation from biomart, using dataset: hsapiens_gene_ensembl
2022-05-23 13:34:51,617 R2G          INFO     Formatting data ...


/tmp/ipykernel_29552/570429411.py:143: RuntimeWarning: invalid value encountered in true_divide
  S = (scores - scores.min()) / (scores.max() - scores.min())
/tmp/ipykernel_29552/570429411.py:151: RuntimeWarning: invalid value encountered in true_divide
  S = (scores - scores.min()) / (scores.max() - scores.min())


2022-05-23 13:35:13,872 R2G          INFO     Writing data to: /staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/bb_files_final/FigR.links.all.bed
2022-05-23 13:35:14,309 R2G          INFO     Writing data to: /staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/bb_files_final/FigR.links.all.bb
2022-05-23 13:35:17,585 R2G          INFO     Downloading gene annotation from biomart, using dataset: hsapiens_gene_ensembl
2022-05-23 13:35:18,122 R2G          INFO     Formatting data ...


...

2022-05-23 13:36:54,840 R2G          INFO     Writing data to: /staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/bb_files_final/GRaNIE.links.all.bed
2022-05-23 13:36:56,976 R2G          INFO     Writing data to: /staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/bb_files_final/GRaNIE.links.all.bb
2022-05-23 13:37:03,508 R2G          INFO     Downloading gene annotation from biomart, using dataset: hsapiens_gene_ensembl
2022-05-23 13:37:04,247 R2G          INFO     Formatting data ...


/tmp/ipykernel_29552/570429411.py:143: RuntimeWarning: invalid value encountered in true_divide
  S = (scores - scores.min()) / (scores.max() - scores.min())
/tmp/ipykernel_29552/570429411.py:151: RuntimeWarning: invalid value encountered in true_divide
  S = (scores - scores.min()) / (scores.max() - scores.min())


2022-05-23 13:38:21,107 R2G          INFO     Writing data to: /staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/bb_files_final/Scenicplus-importance.links.all.bed
2022-05-23 13:38:22,743 R2G          INFO     Writing data to: /staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/bb_files_final/Scenicplus-importance.links.all.bb
2022-05-23 13:38:28,222 R2G          INFO     Downloading gene annotation from biomart, using dataset: hsapiens_gene_ensembl
2022-05-23 13:38:28,760 R2G          INFO     Formatting data ...


/tmp/ipykernel_29552/570429411.py:143: RuntimeWarning: invalid value encountered in true_divide
  S = (scores - scores.min()) / (scores.max() - scores.min())
/tmp/ipykernel_29552/570429411.py:151: RuntimeWarning: invalid value encountered in true_divide
  S = (scores - scores.min()) / (scores.max() - scores.min())


2022-05-23 13:39:48,700 R2G          INFO     Writing data to: /staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/bb_files_final/Scenicplus-rho.links.all.bed
2022-05-23 13:39:50,382 R2G          INFO     Writing data to: /staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/bb_files_final/Scenicplus-rho.links.all.bb


In [23]:
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
cell_lines=['Scenicplus-importance_links_all', 'Scenicplus-rho_links_all']
color_dict={'Scenicplus-importance_links_all':'Blues', 'Scenicplus-rho_links_all':'Reds'}
save_path = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/bb_files_final/'
species = 'hsapiens'
assembly = 'hg38'
path_bedToBigBed = '/data/leuven/software/biomed/haswell_centos7/2018a/software/Kent_tools/20190730-linux.x86_64/bin/'
biomart_host = 'http://oct2016.archive.ensembl.org/'
path='/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/formatted/'
for cell_line in cell_lines:
    files = [f for f in listdir(path) if isfile(join(path, f))]
    file = [f for f in files if cell_line in f]
    hic_data = pd.read_csv(path+file[0], sep='\t')
    hic_data.columns = ['Chromosome', 'Start', 'End', 'target', 'importance', 'TSS', 'Strand', 'Distance', 'region']
    hic_data['rho'] = hic_data['importance']
    hic_data = hic_data[['target', 'region', 'importance', 'rho', 'Distance']]
    hic_data = hic_data[hic_data['region'].isin(scplus_obj.region_names)]
    scplus_obj.uns['region_to_gene'] = hic_data
    r2g_data = export_to_UCSC_interact_hic(scplus_obj,
                    species,
                    os.path.join(save_path,cell_line+'.links.all.bed'),
                    path_bedToBigBed=path_bedToBigBed,
                    bigbed_outfile=os.path.join(save_path,cell_line+'.links.all.bb'),
                    region_to_gene_key='region_to_gene',
                    pbm_host=biomart_host,
                    assembly=assembly,
                    ucsc_track_name='R2G',
                    ucsc_description='SCENIC+ region to gene links',
                    cmap_neg='Reds',
                    cmap_pos=color_dict[cell_line],
                    key_for_color='importance',
                    scale_by_gene=True,
                    subset_for_eRegulons_regions=False,
                    eRegulons_key='eRegulons')


2022-05-23 16:33:13,303 R2G          INFO     Downloading gene annotation from biomart, using dataset: hsapiens_gene_ensembl
2022-05-23 16:33:15,676 R2G          INFO     Formatting data ...


/tmp/ipykernel_29552/570429411.py:143: RuntimeWarning: invalid value encountered in true_divide
  S = (scores - scores.min()) / (scores.max() - scores.min())


KeyboardInterrupt: 

In [64]:



#from scenicplus.enhancer_to_gene import export_to_UCSC_interact 


2022-01-28 15:50:47,701 R2G          INFO     Downloading gene annotation from biomart, using dataset: hsapiens_gene_ensembl
2022-01-28 15:50:48,195 R2G          INFO     Formatting data ...


/opt/venv/lib/python3.8/site-packages/pybiomart/dataset.py:269: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  result = pd.read_csv(StringIO(response.text), sep='\t')


2022-01-28 15:56:13,377 R2G          INFO     Writing data to: /staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/bb_files/HepG2.hic.all.bed
2022-01-28 15:56:27,256 R2G          INFO     Writing data to: /staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/bb_files/HepG2.hic.all.notscaled.bb


In [60]:
hic_data 

,target,region,importance,rho,Distance
10,ZNF493,chr19:21625214-21625715,13.353646,13.353646,228095
12,ZNF493,chr19:21482852-21483353,13.769301,13.769301,85733
13,ZNF493,chr19:21483548-21484049,13.769301,13.769301,86429
15,ZNF493,chr19:21507880-21508381,14.197148,14.197148,110761
24,ZNF493,chr19:21474600-21475101,19.167118,19.167118,77481
...,...,...,...,...,...
2576819,PRR23D2,chr8:27616977-27617478,2559.157700,2559.157700,-19835564
2576825,FAM90A8P,chr8:30469420-30469921,2584.061000,2584.061000,22731033
2576839,GRAPL,chr17:19021510-19022011,3444.935300,3444.935300,-106025
2576844,SLX1A,chr16:22170680-22171181,3885.789000,3885.789000,-8023195


In [62]:
eRegulon_regions = list(set(flatten_list([ereg.target_regions for ereg in scplus_obj.uns['eRegulons']])))
hic_data = hic_data[hic_data['region'].isin(eRegulon_regions)]
scplus_obj.uns['region_to_gene'] = hic_data
r2g_data = export_to_UCSC_interact_hic(scplus_obj,
                    species,
                    os.path.join(save_path,'HepG2.hic.eGRN.bed'),
                    path_bedToBigBed=path_bedToBigBed,
                    bigbed_outfile=os.path.join(save_path,'HepG2.hic.eGRN.notscaled.bb'),
                    region_to_gene_key='region_to_gene',
                    pbm_host=biomart_host,
                    assembly=assembly,
                    ucsc_track_name='R2G',
                    ucsc_description='SCENIC+ region to gene links',
                    cmap_neg='Reds',
                    cmap_pos='Greens',
                    key_for_color='importance',
                    scale_by_gene=False,
                    subset_for_eRegulons_regions=False,
                    eRegulons_key='eRegulons')

2022-01-28 15:48:02,833 R2G          INFO     Downloading gene annotation from biomart, using dataset: hsapiens_gene_ensembl
2022-01-28 15:48:03,532 R2G          INFO     Formatting data ...


/opt/venv/lib/python3.8/site-packages/pybiomart/dataset.py:269: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  result = pd.read_csv(StringIO(response.text), sep='\t')


2022-01-28 15:49:45,835 R2G          INFO     Writing data to: /staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/bb_files/HepG2.hic.eGRN.bed
2022-01-28 15:49:49,890 R2G          INFO     Writing data to: /staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/data/HiC/bb_files/HepG2.hic.eGRN.notscaled.bb


In [ ]:
# For the downstream analyses

#outDir = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/scenicplus_final/'
outDir = '/data/users/cbravo/DPCL/scenicplus_final_autoreg/'
import pickle
infile = open(outDir+'scplus_obj.pkl', 'rb')
scplus_obj = pickle.load(infile)
infile.close()

import pickle
infile = open('/data/users/cbravo/DPCL/scenicplus_final_autoreg/genie3/region_ranking.pkl', 'rb')
region_ranking = pickle.load(infile)
infile.close()

import pickle
infile = open('/data/users/cbravo/DPCL/scenicplus_final_autoreg/genie3/gene_ranking.pkl', 'rb')
gene_ranking = pickle.load(infile)
infile.close()

from scenicplus.wrappers.run_scenicplus import *
run_scenicplus_genie3(scplus_obj,
    variable = ['ACC_Cell_type'],
    species = 'hsapiens',
    assembly = 'hg38',
    tf_file = '/data/users/cbravo/resources/allTFs_hg38.txt',
    save_path = '/data/users/cbravo/DPCL/scenicplus_final_autoreg/genie3/',
    biomart_host = 'http://oct2016.archive.ensembl.org/',
    upstream = [1000, 150000],
    downstream = [1000, 150000],   
    calculate_TF_eGRN_correlation = False,
    calculate_DEGs_DARs = True,
    export_to_loom_file = True,
    export_to_UCSC_file = True,
    region_ranking=region_ranking,
    gene_ranking=gene_ranking,
    tree_structure = ('DPCL', 'SCENIC+', 'genie3'),
    path_bedToBigBed = '/media/data/users/cbravo/software/KENT/',
    n_cpu = 20,
    _temp_dir = '/media/data/users/cbravo/ray_spill'
    )

from scenicplus.wrappers.run_scenicplus import *
run_scenicplus_genie3(scplus_obj,
    variable = ['ACC_Cell_type'],
    species = 'hsapiens',
    assembly = 'hg38',
    tf_file = '/staging/leuven/stg_00002/lcb/cflerin/resources/allTFs_hg38.txt',
    save_path = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/scenicplus_final_autoreg/genie3/',
    biomart_host = 'http://oct2016.archive.ensembl.org/',
    upstream = [1000, 150000],
    downstream = [1000, 150000],   
    calculate_TF_eGRN_correlation = False,
    calculate_DEGs_DARs = True,
    export_to_loom_file = True,
    export_to_UCSC_file = True,
    tree_structure = ('DPCL', 'SCENIC+', 'genie3'),
    path_bedToBigBed = '/data/leuven/software/biomed/haswell_centos7/2018a/software/Kent_tools/20190730-linux.x86_64/bin/',
    n_cpu = 14,
    _temp_dir = '/scratch/leuven/313/vsc31305/ray_spill'
    )

In [ ]:
from scenicplus.scenicplus_class import SCENICPLUS, create_SCENICPLUS_object
from scenicplus.preprocessing.filtering import *
from scenicplus.cistromes import *
from scenicplus.enhancer_to_gene import get_search_space, calculate_regions_to_genes_relationships, RF_KWARGS
from scenicplus.enhancer_to_gene import export_to_UCSC_interact 
from scenicplus.utils import format_egrns, export_eRegulons
from scenicplus.eregulon_enrichment import *
from scenicplus.TF_to_gene import *
from scenicplus.grn_builder.gsea_approach import build_grn
from scenicplus.dimensionality_reduction import *
from scenicplus.RSS import *
from scenicplus.diff_features import *
from scenicplus.loom import *
from typing import Dict, List, Mapping, Optional, Sequence
import os
import dill
import time

def run_scenicplus_genie3(scplus_obj: 'SCENICPLUS',
    variable: List[str],
    species: str,
    assembly: str,
    tf_file: str,
    save_path: str,
    biomart_host: Optional[str] = 'http://www.ensembl.org',
    upstream: Optional[List] = [1000, 150000],
    downstream: Optional[List] = [1000, 150000],
    region_ranking: Optional['CisTopicImputedFeatures'] = None,
    gene_ranking: Optional['CisTopicImputedFeatures'] = None,
    calculate_TF_eGRN_correlation: Optional[bool] = True,
    calculate_DEGs_DARs: Optional[bool] = True,
    export_to_loom_file: Optional[bool] = True,
    export_to_UCSC_file: Optional[bool] = True,
    tree_structure: Sequence[str] = (),
    path_bedToBigBed: Optional[str] = None,
    n_cpu: Optional[int] = 1,
    _temp_dir: Optional[str] = '/scratch/leuven/313/vsc31305/ray_spill'
    ):
    """
    Wrapper to run SCENIC+
    
    Parameters
    ---------
    scplus_obj: `class::SCENICPLUS`
        A SCENICPLUS object.
    variables: List[str]
        Variables to use for RSS, TF-eGRN correlation and markers.
    species: str
        Species from which data comes from. Possible values: 'hsapiens', 'mmusculus', 'dmelanogaster'
    assembly: str
        Genome assembly to which the data was mapped. Possible values: 'hg38'
    tf_file: str
        Path to file containing genes that are TFs
    save_path: str
        Folder in which results will be saved
    biomart_host: str, optional
        Path to biomart host. Make sure that the host matches your genome assembly
    upstream: str, optional
        Upstream space to use for region to gene relationships
    downstream: str, optional
        Upstream space to use for region to gene relationships
    region_ranking: `class::CisTopicImputedFeatures`, optional
        Precomputed region ranking
    gene_ranking: `class::CisTopicImputedFeatures`, optional
        Precomputed gene ranking
    calculate_TF_eGRN_correlation: bool, optional
        Whether to calculate the TF-eGRN correlation based on the variables
    calculate_DEGs_DARs: bool, optional
        Whether to calculate DARs/DEGs based on the variables
    export_to_loom_file: bool, optional
        Whether to export data to loom files (gene based/region based)
    export_to_UCSC_file: bool, optional
        Whether to export region-to-gene links and eregulons to bed files
    tree_structure: sequence, optional
        Tree structure for loom files
    path_bedToBigBed: str, optional
        Path to convert bed files to big bed when exporting to UCSC (required if files are meant to be
        used in a hub)
    n_cpu: int, optional
        Number of cores to use
    _temp_dir: str, optional
        Temporary directory for ray
    """
    
    # Create logger
    level = logging.INFO
    log_format = '%(asctime)s %(name)-12s %(levelname)-8s %(message)s'
    handlers = [logging.StreamHandler(stream=sys.stdout)]
    logging.basicConfig(level=level, format=log_format, handlers=handlers)
    log = logging.getLogger('SCENIC+_wrapper')
    
    start_time = time.time()
    
    check_folder = os.path.isdir(save_path)
    if not check_folder:
        os.makedirs(save_path)
        log.info("Created folder : "+ save_path)

    else:
        log.info(save_path + " folder already exists.")
    
    if 'Cistromes' not in scplus_obj.uns.keys():
        log.info('Merging cistromes')
        merge_cistromes(scplus_obj)
    
    
    if 'search_space' not in scplus_obj.uns.keys():
        log.info('Getting search space')
        get_search_space(scplus_obj,
                     biomart_host = biomart_host,
                     species = species,
                     assembly = assembly, 
                     upstream = upstream,
                     downstream = downstream)
                 
    if 'region_to_gene' not in scplus_obj.uns.keys():
        log.info('Inferring region to gene relationships')
        calculate_regions_to_genes_relationships(scplus_obj, 
                        ray_n_cpu = n_cpu, 
                        _temp_dir = _temp_dir,
                        importance_scoring_method = 'RF',
                        importance_scoring_kwargs = RF_KWARGS)
                        
    if 'TF2G_adj' not in scplus_obj.uns.keys():
        log.info('Inferring TF to gene relationships')
        calculate_TFs_to_genes_relationships(scplus_obj, 
                        tf_file = tf_file,
                        ray_n_cpu = n_cpu, 
                        method = 'GBM',
                        _temp_dir = _temp_dir,
                        key= 'TF2G_adj')
                        
    if 'eRegulons' not in scplus_obj.uns.keys():
        log.info('Build eGRN')
        build_grn(scplus_obj,
                 min_target_genes = 10,
                 adj_pval_thr = 1,
                 min_regions_per_gene = 0,
                 quantiles = (0.85, 0.90, 0.95),
                 top_n_regionTogenes_per_gene = (5, 10, 15),
                 top_n_regionTogenes_per_region = (),
                 binarize_using_basc = True,
                 rho_dichotomize_tf2g = True,
                 rho_dichotomize_r2g = True,
                 rho_dichotomize_eregulon = True,
                 rho_threshold = 0.05,
                 keep_extended_motif_annot = True,
                 merge_eRegulons = True, 
                 order_regions_to_genes_by = 'importance',
                 order_TFs_to_genes_by = 'importance',
                 key_added = 'eRegulons',
                 cistromes_key = 'Unfiltered',
                 disable_tqdm = True, 
                 ray_n_cpu = n_cpu,
                 _temp_dir = _temp_dir)

    if 'eRegulon_metadata' not in scplus_obj.uns.keys():
        log.info('Formatting eGRNs')
        format_egrns(scplus_obj,
                      eregulons_key = 'eRegulons',
                      TF2G_key = 'TF2G_adj',
                      key_added = 'eRegulon_metadata')


    if 'eRegulon_signatures' not in scplus_obj.uns.keys():
        log.info('Converting eGRNs to signatures')
        get_eRegulons_as_signatures(scplus_obj,
                                     eRegulon_metadata_key='eRegulon_metadata', 
                                     key_added='eRegulon_signatures')

#if 'eRegulon_AUC' not in scplus_obj.uns.keys():
    log.info('Calculating eGRNs AUC')
    if region_ranking is None:
        log.info('Calculating region ranking')
        region_ranking = make_rankings(scplus_obj, target='region')
        with open(os.path.join(save_path,'region_ranking.pkl'), 'wb') as f:
            dill.dump(region_ranking, f)
    log.info('Calculating eGRNs region based AUC')
    score_eRegulons(scplus_obj,
            ranking = region_ranking,
            eRegulon_signatures_key = 'eRegulon_signatures',
            key_added = 'eRegulon_AUC', 
            enrichment_type= 'region',
            auc_threshold = 0.05,
            normalize = False,
            n_cpu = n_cpu)
    if gene_ranking is None:
        log.info('Calculating gene ranking')
        gene_ranking = make_rankings(scplus_obj, target='gene')
        with open(os.path.join(save_path,'gene_ranking.pkl'), 'wb') as f:
            dill.dump(gene_ranking, f)
    log.info('Calculating eGRNs gene based AUC')
    score_eRegulons(scplus_obj,
            gene_ranking,
            eRegulon_signatures_key = 'eRegulon_signatures',
            key_added = 'eRegulon_AUC', 
            enrichment_type = 'gene',
            auc_threshold = 0.05,
            normalize= False,
            n_cpu = n_cpu)
                
    if calculate_TF_eGRN_correlation is True:
        log.info('Calculating TF-eGRNs AUC correlation')
        for var in variable:
            generate_pseudobulks(scplus_obj, 
                             variable = var,
                             auc_key = 'eRegulon_AUC',
                             signature_key = 'Gene_based',
                             nr_cells = 5,
                             nr_pseudobulks = 100,
                             seed=555)
            generate_pseudobulks(scplus_obj, 
                                     variable = var,
                                     auc_key = 'eRegulon_AUC',
                                     signature_key = 'Region_based',
                                     nr_cells = 5,
                                     nr_pseudobulks = 100,
                                     seed=555)
            TF_cistrome_correlation(scplus_obj,
                            variable = var, 
                            auc_key = 'eRegulon_AUC',
                            signature_key = 'Gene_based',
                            out_key = var+'_eGRN_gene_based')
            TF_cistrome_correlation(scplus_obj,
                                    variable = var, 
                                    auc_key = 'eRegulon_AUC',
                                    signature_key = 'Region_based',
                                    out_key = var+'_eGRN_region_based')
                                
#if 'eRegulon_AUC_thresholds' not in scplus_obj.uns.keys():
    log.info('Binarizing eGRNs AUC')
    binarize_AUC(scplus_obj, 
         auc_key='eRegulon_AUC',
         out_key='eRegulon_AUC_thresholds',
         signature_keys=['Gene_based', 'Region_based'],
         n_cpu=n_cpu)

#if 'eRegulons_UMAP' not in scplus_obj.dr_cell.keys():
    log.info('Making eGRNs AUC UMAP')
    run_eRegulons_umap(scplus_obj,
               scale=True, signature_keys=['Gene_based', 'Region_based'])
#if 'eRegulons_tSNE' not in scplus_obj.dr_cell.keys():
    log.info('Making eGRNs AUC tSNE')
    run_eRegulons_tsne(scplus_obj,
               scale=True, signature_keys=['Gene_based', 'Region_based'])

#if 'RSS' not in scplus_obj.uns.keys():
    log.info('Calculating eRSS')
    for var in variable:
        regulon_specificity_scores(scplus_obj, 
                     var,
                     signature_keys=['Gene_based'],
                     out_key_suffix='_gene_based',
                     scale=False)
        regulon_specificity_scores(scplus_obj, 
                     var,
                     signature_keys=['Region_based'],
                     out_key_suffix='_region_based',
                         scale=False)
                         
    if calculate_DEGs_DARs is True:
        log.info('Calculating DEGs/DARs')
        for var in variable:
            get_differential_features(scplus_obj, var, use_hvg = True, contrast_type = ['DEGs', 'DARs'])
            
    if export_to_loom_file is True:
        log.info('Exporting to loom file')
        export_to_loom(scplus_obj, 
               signature_key = 'Gene_based',
               tree_structure = tree_structure,
               title =  'Gene based eGRN',
               nomenclature = assembly,
               out_fname=os.path.join(save_path,'SCENIC+_gene_based.loom'))
        export_to_loom(scplus_obj, 
               signature_key = 'Region_based',
               tree_structure = tree_structure,
               title =  'Region based eGRN',
               nomenclature = assembly,
               out_fname=os.path.join(save_path,'SCENIC+_region_based.loom'))
               
    if export_to_UCSC_file is True:
        log.info('Exporting to UCSC')
        r2g_data = export_to_UCSC_interact(scplus_obj,
                            species,
                            os.path.join(save_path,'r2g.rho.bed'),
                            path_bedToBigBed=path_bedToBigBed,
                            bigbed_outfile=os.path.join(save_path,'r2g.rho.bb'),
                            region_to_gene_key='region_to_gene',
                            pbm_host=biomart_host,
                            assembly=assembly,
                            ucsc_track_name='R2G',
                            ucsc_description='SCENIC+ region to gene links',
                            cmap_neg='Reds',
                            cmap_pos='Greens',
                            key_for_color='rho',
                            scale_by_gene=False,
                            subset_for_eRegulons_regions=True,
                            eRegulons_key='eRegulons')
        r2g_data = export_to_UCSC_interact(scplus_obj,
                            species,
                            os.path.join(save_path,'r2g.importance.bed'),
                            path_bedToBigBed=path_bedToBigBed,
                            bigbed_outfile=os.path.join(save_path,'r2g.importance.bb'),
                            region_to_gene_key='region_to_gene',
                            pbm_host=biomart_host,
                            assembly=assembly,
                            ucsc_track_name='R2G',
                            ucsc_description='SCENIC+ region to gene links',
                            cmap_neg='Reds',
                            cmap_pos='Greens',
                            key_for_color='importance',
                            scale_by_gene=True,
                            subset_for_eRegulons_regions=True,
                            eRegulons_key='eRegulons')
        regions = export_eRegulons(scplus_obj,
                os.path.join(save_path,'eRegulons.bed'),
                assembly,
                bigbed_outfile = os.path.join(save_path,'eRegulons.bb'),
                eRegulon_metadata_key = 'eRegulon_metadata',
                eRegulon_signature_key = 'eRegulon_signatures',
                path_bedToBigBed=path_bedToBigBed)
        
    log.info('Saving object')         
    with open(os.path.join(save_path,'scplus_obj_genie3.pkl'), 'wb') as f:
        dill.dump(scplus_obj, f)

    log.info('Finished! Took {} minutes'.format((time.time() - start_time)/60))       
    
    